In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, multilabel_confusion_matrix
import models
import process_data
import analysis
import train_ecg
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
#setup
BATCH_SIZE = 256
LEARNING_RATE = 0.0005
NUM_EPOCHS = 25

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()


In [3]:
%%time
train_loader, val_loader = process_data.load_data(batch_size=BATCH_SIZE, smote=False)

Wall time: 3.03 s


In [4]:
# Confusion matrix
def confusion_matrix(cm, y_true):
    df_cm = pd.DataFrame(cm, columns=np.unique(y_true), index = np.unique(y_true))
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (10,7))
    sn.set(font_scale=1.4)#for label size
    sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16})# font size


In [ ]:
embeds,y = analysis.get_embeddings(cnetmodel, train_loader)
coordinates = analysis.get_embeddings(embeds)
analysis.plot_tsne(coordinates, y, 5)

In [1]:
# BiRNN model

In [23]:
from torch.nn.init import xavier_uniform_

class BiRNN(nn.Module):
    def __init__(self, is_lstm=True):
        super().__init__()
        self.dropout = nn.Dropout(0.2)
        self.encoder = nn.LSTM(input_size=187, hidden_size=128, bidirectional=True, dropout=0.1, num_layers=2)
        if not is_lstm:
            self.encoder = nn.GRU(input_size=187, hidden_size=128, bidirectional=True, dropout=0.1, num_layers=2)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 5)
        
    def forward(self, x):
        x = x.reshape(-1, 1, 187)
        output, _ = self.encoder(x)
        biout = output[:,:,:128] + output[:,:,128:]
        biout = biout.reshape(-1, 128)
        o = F.relu(self.fc1(biout))
        o = self.fc2(o)
        return o

model = BiRNN(is_lstm=False).to(device)
criterion = nn.CrossEntropyLoss()


In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
train_ecg.train(model, train_loader, val_loader, 20, criterion, optimizer)

Epoch: 1 	 Training Loss: 0.061097
Epoch: 1 	 Validation f: 0.87, acc: 0.98
avg metrics : (0.9901333820573726, 0.9783698365517565, 0.8281589045391291, 0.9904720831166658, 0.9198749767857477)
Epoch: 2 	 Training Loss: 0.058867
Epoch: 2 	 Validation f: 0.87, acc: 0.98
avg metrics : (0.9901516535720811, 0.9792041000758418, 0.8311011378955507, 0.9898212954822292, 0.9143048808194328)
Epoch: 3 	 Training Loss: 0.058106
Epoch: 3 	 Validation f: 0.87, acc: 0.98
avg metrics : (0.9903343687191669, 0.9784727937601734, 0.8236345760003836, 0.9909983690502882, 0.9251635852596362)
Epoch: 4 	 Training Loss: 0.059459
Epoch: 4 	 Validation f: 0.87, acc: 0.98
avg metrics : (0.9904439978074183, 0.9795282524830764, 0.827738085333138, 0.9903937209512372, 0.924252964974566)
Epoch: 5 	 Training Loss: 0.057119
Epoch: 5 	 Validation f: 0.87, acc: 0.98
avg metrics : (0.9905718984103783, 0.9787712554820763, 0.8289853433244405, 0.991262750857723, 0.9351672358998273)
Epoch: 6 	 Training Loss: 0.056240
Epoch: 6 	 Va

In [22]:
import scikitplot as skplt
